In [1]:
from orbit_generation.experiment import generate_parameter_sets, paralelize_notebook_experiment, create_experiments_json

Parameters

In [2]:
params = {
    # Data
    'data_used': 'EM_N_fix_1500',
    'families_to_discard': [0, 5, 10, 20, 30],
    'seq_len': 100,
    'feature_dim': 7,
    
    # Training
    'epochs': 50,
    'val_split': 0.05,
    'batch_size': 32,
    'lr': 0.001,
    
    # Model
    'model_name': ['vae_conv5_legit', 'inception_time_wp_vae'],
    'latent_dim': [2, 6],
    'beta': [0.001, 0.2, 0.5, 1, 1.5, 2, 10],
    
    # Convergence
    'max_iter_convergence': 20,
    'input_seq_len_convergence': 1,
    
    # Evaluation
    'samples_to_generate': 100,
    'distance_metric': 'euclidean'
}

In [3]:
model_specific_params = {
    'vae_conv5_legit': {
        'dropout_rate': 0.2
    },
    'inception_time_wp_vae': {
        'n_filters': 32,
        'kernel_sizes': [3, 7, 13],
        'bottleneck_channels': 32
    }
}

Parameter Set

In [4]:
parameter_sets = generate_parameter_sets(params, model_specific_params)

In [5]:
len(parameter_sets)

140

In [6]:
parameter_sets

[{'data_used': 'EM_N_fix_1500',
  'families_to_discard': 0,
  'seq_len': 100,
  'feature_dim': 7,
  'epochs': 50,
  'val_split': 0.05,
  'batch_size': 32,
  'lr': 0.001,
  'model_name': 'vae_conv5_legit',
  'latent_dim': 2,
  'max_iter_convergence': 20,
  'input_seq_len_convergence': 1,
  'samples_to_generate': 100,
  'distance_metric': 'euclidean',
  'model_kwargs': {'dropout_rate': 0.2, 'beta': 0.001}},
 {'data_used': 'EM_N_fix_1500',
  'families_to_discard': 0,
  'seq_len': 100,
  'feature_dim': 7,
  'epochs': 50,
  'val_split': 0.05,
  'batch_size': 32,
  'lr': 0.001,
  'model_name': 'vae_conv5_legit',
  'latent_dim': 2,
  'max_iter_convergence': 20,
  'input_seq_len_convergence': 1,
  'samples_to_generate': 100,
  'distance_metric': 'euclidean',
  'model_kwargs': {'dropout_rate': 0.2, 'beta': 0.2}},
 {'data_used': 'EM_N_fix_1500',
  'families_to_discard': 0,
  'seq_len': 100,
  'feature_dim': 7,
  'epochs': 50,
  'val_split': 0.05,
  'batch_size': 32,
  'lr': 0.001,
  'model_name'

In [7]:
extra_parameters = {
    'want_to_train': False,  # Set this parameter to False if you want to load the model from the folder
    'want_to_generate': True,  # Set this parameter to False if you want to load the generation from the folder
    'want_to_get_cluster_metrics': False,  # Set this parameter to False if you don't want to calculate clustering metrics
    'want_to_perform_convergence': True,  # Set this parameter to False if you don't want to perform convergence
    'want_to_discover': False
}

In [ ]:
notebook_to_execute = '03_01_generative_discovery.ipynb'
output_dir = "../experiments/03_01_generative_discovery"
checkpoint_file = '../experiments/experiment_checkpoint.json'

paralelize_notebook_experiment(
    parameter_sets,
    notebook_to_execute=notebook_to_execute,
    output_dir=output_dir,
    checkpoint_file=checkpoint_file,
    extra_parameters=extra_parameters,
    max_workers=2
)